In [17]:
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Conv3D, MaxPooling3D
from tensorflow.python.keras.layers import BatchNormalization
from tensorflow.python.keras.layers import GlobalMaxPool2D
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import Reshape, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.python.keras.utils import np_utils, generic_utils

import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import cv2
from sklearn import preprocessing

In [18]:
nb_classes = 2
dropout_rate = 0.2
classes = ['NotCrash', 'Crash']

model = Sequential()

strides = (1,1,1)
kernel_size = (3, 3, 3)

model.add(Conv3D(32, kernel_size, strides=strides, activation='relu', padding='same', input_shape=(30, 96, 144, 3)))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(64, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(128, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(1, 2, 2)))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())
#model.add(Dropout(dropout_rate))

model.add(Conv3D(256, kernel_size, strides=strides, activation='relu',padding='same'))
print(model.output_shape)
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(1,12,18)))
print(model.output_shape)

model.add(Reshape((30, 256)))
print(model.output_shape)
model.add(LSTM(256, return_sequences=True))
print(model.output_shape)
model.add(LSTM(256))
print(model.output_shape)

model.add(Dense(256, activation='relu'))
print(model.output_shape)
#model.add(Dropout(dropout_rate))

model.add(Dense(nb_classes, activation='sigmoid'))
print(model.output_shape)

(None, 30, 96, 144, 32)
(None, 30, 48, 72, 32)
(None, 30, 48, 72, 64)
(None, 30, 24, 36, 64)
(None, 30, 24, 36, 128)
(None, 30, 12, 18, 128)
(None, 30, 12, 18, 256)
(None, 30, 12, 18, 256)
(None, 30, 1, 1, 256)
(None, 30, 256)
(None, 30, 256)
(None, 256)
(None, 256)
(None, 2)


In [19]:
def show(img, title="img"):
    cv2.imshow(title, img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [20]:
# Resize frames

def resize_frame(frame, size):
    frame = img.imread(frame)
    frame = cv2.resize(frame, size)
    return frame

In [21]:
from video_datagen import VideoDataGenerator

gen = VideoDataGenerator()

# RGB MODEL TRAINING

In [22]:
sgd = SGD(lr=0.001)
model.compile(optimizer=sgd, loss= 'binary_crossentropy', metrics=['accuracy'])

In [7]:
training_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/train'
validation_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/validation'

batchsize = 1
training_set = gen.flow_from_directory(training_path, target_size=(128, 192), batch_size=batchsize, clip_size=30)
validation_set = gen.flow_from_directory(validation_path, target_size=(128, 192), batch_size=4, clip_size=30)

Found 873 videos belonging to 2 classes.
Found 78 videos belonging to 2 classes.


In [6]:
model.load_weights('best.h5')

In [8]:
import tensorflow as tf

tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.33)

#tf.Session(config=tf.ConfigProto(gpu_options=gpu_options,allow_soft_placement=True)

per_process_gpu_memory_fraction: 0.33

In [ ]:
training_history_rgb2 = model.fit_generator(training_set, use_multiprocessing=True,
                                       epochs =20,validation_data = validation_set , verbose = 1 , validation_steps = 20,
                                       steps_per_epoch=872)

Epoch 1/20
872/872 [==============================] - 587s 673ms/step - loss: 0.6672 - accuracy: 0.6061 - val_loss: 0.6959 - val_accuracy: 0.5769
Epoch 2/20
872/872 [==============================] - 588s 675ms/step - loss: 0.6417 - accuracy: 0.5900 - val_loss: 0.6511 - val_accuracy: 0.5705
Epoch 3/20
872/872 [==============================] - 586s 672ms/step - loss: 0.6166 - accuracy: 0.6044 - val_loss: 0.6445 - val_accuracy: 0.5705
Epoch 4/20
872/872 [==============================] - 585s 671ms/step - loss: 0.6094 - accuracy: 0.6181 - val_loss: 0.6628 - val_accuracy: 0.5897
Epoch 5/20
872/872 [==============================] - 581s 666ms/step - loss: 0.6079 - accuracy: 0.6164 - val_loss: 0.7907 - val_accuracy: 0.5449
Epoch 6/20
872/872 [==============================] - 579s 664ms/step - loss: 0.6081 - accuracy: 0.6055 - val_loss: 0.6461 - val_accuracy: 0.5769
Epoch 7/20
872/872 [==============================] - 581s 666ms/step - loss: 0.5948 - accuracy: 0.6216 - val_loss: 0.6491 -

In [9]:
model.save_weights('model2-sgd-0001-20epochs-orgsize.h5')

In [10]:
import pickle

pickling_on = open("history_rgb_2orgsize.pickle","wb")
pickle.dump(training_history_rgb2.history, pickling_on)
pickling_on.close()

In [ ]:
training_history_flow_sized = model.fit_generator(training_set, use_multiprocessing=True,
                                       epochs =20,validation_data = validation_set , verbose = 1 , validation_steps = 1,
                                       steps_per_epoch=872//2)

In [18]:
model.save_weights('model-flow-sgd-0001-50epochs.h5')

# Testing on test dataset

In [23]:
model.load_weights('best.h5')

In [24]:
testing_path = '/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test'

testing_set = gen.flow_from_directory(testing_path, target_size=(96, 144), batch_size=1, clip_size=30)

Found 48 videos belonging to 2 classes.


In [25]:
# calculating test result accuracy

test_size = len(os.listdir(f'{testing_path}/Crash')) + len(os.listdir(f'{testing_path}/NotCrash'))
iterations = 0
accuracy = []
while True:
    inst = next(testing_set)
    score = model.evaluate(inst[0], inst[1], verbose = 0)
    accuracy.append(score[1])
    iterations += 1
    
    if iterations == test_size:
        break
print(sum(accuracy)/len(accuracy))

0.7291666666666666


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 96, 144, 32)   2624      
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 96, 144, 32)   128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 30, 48, 72, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 48, 72, 64)    55360     
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 48, 72, 64)    256       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 24, 36, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 30, 24, 36, 128)   2

In [10]:
import pickle

In [31]:
model.classes

AttributeError: 'Sequential' object has no attribute 'classes'

In [28]:
to_predict = []

model.load_weights('best.h5')
num_frames = 0
cap = cv2.VideoCapture('/home/siddhant/Codes/Accident Detection/accident3.mp4')

#cap.set(12, 50)
#cap.set(6, 10)
#cap.set(cv2.CAP_PROP_FPS, 10)

preds = []

classe = ''
import time
fps = 30
counter  = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_cp = cv2.resize(frame, (144, 96))
    
    if counter%1 == 0:
        to_predict.append(frame_cp)
    counter += 1
    predict = 0
    if len(to_predict) == 30:

        frame_to_predict = [[]]
        frame_to_predict[0] = np.array(to_predict, dtype=np.float32)


        predict = model.predict(np.array(frame_to_predict))
        classe = classes[np.argmax(predict)]
        if np.amax(predict) > 0.60:
            print('Class = ',classe, 'Precision = ', np.amax(predict)*100,'%')
            preds.append(np.argmax(predict))
           # with open('gesture.pkl','wb') as f:
                #pickle.dump(np.argmax(predict), f)
        if len(preds) >= 10:
            preds = preds[8:9]

        to_predict = []
        font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, classe, (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255),1,cv2.LINE_AA)
    
    cv2.imshow('Live Accident Detection',frame)
    if cv2.waitKey(int( (1 / int(fps)) * 1000)) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Class =  NotCrash Precision =  60.540771484375 %
Class =  NotCrash Precision =  62.795281410217285 %
Class =  NotCrash Precision =  60.591113567352295 %
Class =  NotCrash Precision =  67.68972277641296 %
Class =  NotCrash Precision =  78.3884048461914 %
Class =  NotCrash Precision =  68.09093356132507 %
Class =  NotCrash Precision =  71.17609977722168 %
Class =  NotCrash Precision =  69.8011040687561 %
Class =  NotCrash Precision =  89.85544443130493 %
Class =  NotCrash Precision =  91.48525595664978 %
Class =  Crash Precision =  73.48454594612122 %
Class =  Crash Precision =  88.89908194541931 %
Class =  Crash Precision =  89.17043805122375 %
Class =  Crash Precision =  67.25341081619263 %
Class =  NotCrash Precision =  92.02522039413452 %
Class =  NotCrash Precision =  91.55519008636475 %
Class =  NotCrash Precision =  94.72222924232483 %
Class =  NotCrash Precision =  95.68073153495789 %
Class =  NotCrash Precision =  95.51900625228882 %
Class =  NotCrash Precision =  93.65037679672

In [69]:
def readSamples(pathToSample, resize_dim = (144, 96)):
    # pathToSample points to dir containing the frames
    frames = os.listdir(pathToSample)
    frames.sort()
    sample = []
    for frame in frames:
        im = cv2.imread(f'{pathToSample}/{frame}')
        im = cv2.resize(im, resize_dim)
        sample.append(im)
    return sample

In [72]:
model.load_weights('model-sgd-0001-20epochs3.h5')

In [81]:
sample = readSamples('/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test/NotCrash/221')

sample = [sample]
sample = np.array(sample, dtype=np.float32)
model.predict(sample)

array([[0.95695245, 0.04283412]], dtype=float32)

In [35]:
np.shape(sample)

(1, 30, 96, 144, 3)

In [74]:
print(os.listdir('/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test/NotCrash/'))
print(os.listdir('/home/siddhant/Datasets/Crash Dataset/Final_Preprocessed/test/Crash/'))

['320', '38', '108', '66', '137', '263', '165', '249', '94', '151', '123', '207', '291', '305', '221', '334', '80', '179', '235', '193', '52', '277', '10', '24']
['9', '452', '153', '605', '331', '36', '361', '544', '88', '140', '23', '75', '422', '114', '269', '498', '376', '437', '223', '514', '284', '62', '391', '574', '208', '101', '238', '467', '254', '406', '166', '315', '179', '193', '346', '49', '127', '529', '300', '559', '483', '589']


In [91]:
import glob

vid_dir = ''

images = glob.glob(os.path.join('/home/siddhant/Codes/Accident Detection/221/','*.png'))

In [92]:
images

['/home/siddhant/Codes/Accident Detection/221/003.png',
 '/home/siddhant/Codes/Accident Detection/221/024.png',
 '/home/siddhant/Codes/Accident Detection/221/025.png',
 '/home/siddhant/Codes/Accident Detection/221/013.png',
 '/home/siddhant/Codes/Accident Detection/221/001.png',
 '/home/siddhant/Codes/Accident Detection/221/004.png',
 '/home/siddhant/Codes/Accident Detection/221/008.png',
 '/home/siddhant/Codes/Accident Detection/221/010.png',
 '/home/siddhant/Codes/Accident Detection/221/000.png',
 '/home/siddhant/Codes/Accident Detection/221/022.png',
 '/home/siddhant/Codes/Accident Detection/221/023.png',
 '/home/siddhant/Codes/Accident Detection/221/007.png',
 '/home/siddhant/Codes/Accident Detection/221/002.png',
 '/home/siddhant/Codes/Accident Detection/221/006.png',
 '/home/siddhant/Codes/Accident Detection/221/009.png',
 '/home/siddhant/Codes/Accident Detection/221/028.png',
 '/home/siddhant/Codes/Accident Detection/221/018.png',
 '/home/siddhant/Codes/Accident Detection/221/01